# Day 0

In [12]:
import pandas as pd
# Python 3.x Utility Functions

import re
import numpy as np
import math
import random
import urllib.request

from collections import Counter, defaultdict, namedtuple, deque
from functools   import lru_cache
from itertools   import permutations, combinations, chain, cycle, product, islice, count as count_from
from heapq       import heappop, heappush

identity = lambda x: x
letters  = 'abcdefghijklmnopqrstuvwxyz'

cat = ''.join

Ø   = frozenset() # Empty set
inf = float('inf')
BIG = 10 ** 999

def Input(day, year=2017):
    "Open this day's input file."
    filename = 'data/advent{}/input{}.txt'.format(year, day)
    try:
        return open(filename)
    except FileNotFoundError:
        return urllib.request.urlopen("http://norvig.com/ipython/" + filename)
    
def array(lines):
    "Convert an iterable of lines into a 2-D array. If lines is a str, do splitlines."
    if isinstance(lines, str): lines = lines.splitlines()
    return [mapt(atom, line.split()) for line in lines]

def atom(token):
    try:
        return int(token)
    except ValueError:
        try:
            return float(token)
        except ValueError:
            return token

################ Functions on Iterables

def first(iterable, default=None): return next(iter(iterable), default)

def first_true(iterable, pred=None, default=None):
    """Returns the first true value in the iterable.
    If no true value is found, returns *default*
    If *pred* is not None, returns the first item
    for which pred(item) is true."""
    # first_true([a,b,c], default=x) --> a or b or c or x
    # first_true([a,b], fn, x) --> a if fn(a) else b if fn(b) else x
    return next(filter(pred, iterable), default)

def nth(iterable, n, default=None):
    "Returns the nth item of iterable, or a default value"
    return next(islice(iterable, n, None), default)

def upto(iterable, maxval):
    "From a monotonically increasing iterable, generate all the values <= maxval."
    # Why <= maxval rather than < maxval? In part because that's how Ruby's upto does it.
    return takewhile(lambda x: x <= maxval, iterable)

def groupby(iterable, key=identity):
    "Return a dict of {key(item): [items...]} grouping all items in iterable by keys."
    groups = defaultdict(list)
    for item in iterable:
        groups[key(item)].append(item)
    return groups

def grouper(iterable, n, fillvalue=None):
    """Collect data into fixed-length chunks:
    grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx"""
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)

def overlapping(iterable, n):
    """Generate all (overlapping) n-element subsequences of iterable.
    overlapping('ABCDEFG', 3) --> ABC BCD CDE DEF EFG"""
    if isinstance(iterable, abc.Sequence):
        yield from (iterable[i:i+n] for i in range(len(iterable) + 1 - n))
    else:
        result = deque(maxlen=n)
        for x in iterable:
            result.append(x)
            if len(result) == n:
                yield tuple(result)
                
def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    return overlapping(iterable, 2)

def sequence(iterable, type=tuple):
    "Coerce iterable to sequence: leave it alone if it is already a sequence, else make it of type."
    return iterable if isinstance(iterable, abc.Sequence) else type(iterable)

def join(iterable, sep=''):
    "Join the items in iterable, converting each to a string first."
    return sep.join(map(str, iterable))
                
def powerset(iterable):
    "Yield all subsets of items."
    items = list(iterable)
    for r in range(len(items)+1):
        for c in combinations(items, r):
            yield c
            
def quantify(iterable, pred=bool):
    "Count how many times the predicate is true."
    return sum(map(pred, iterable))

def shuffled(iterable):
    "Create a new list out of iterable, and shuffle it."
    new = list(iterable)
    random.shuffle(new)
    return new
    
flatten = chain.from_iterable
            
class Set(frozenset):
    "A frozenset, but with a prettier printer."
    def __repr__(self): return '{' + join(sorted(self), ', ') + '}'
    
def canon(items, typ=None):
    "Canonicalize these order-independent items into a hashable canonical form."
    typ = (typ or (cat if isinstance(items, str) else tuple))
    return typ(sorted(items))

def mapt(fn, *args): 
    "Do a map, and make the results into a tuple."
    return tuple(map(fn, *args))
            
################ Math
            
def transpose(matrix): return tuple(zip(*matrix))

def isqrt(n):
    "Integer square root (rounds down)."
    return int(n ** 0.5)

def ints(start, end):
    "The integers from start to end, inclusive. Equivalent to range(start, end+1)"
    return range(start, end + 1)

def floats(start, end, step=1.0):
    "Yields start, start+step, start+2*step, ... up to (maybe including) end."
    m = (1.0 if step >= 0 else -1.0)
    while start * m <= end * m:
        yield start
        start += step
        
def multiply(numbers):
    "Multiply all the numbers together."
    result = 1
    for n in numbers:
        result *= n
    return result

################ 2-D points implemented using (x, y) tuples

def X(point): x, y = point; return x
def Y(point): x, y = point; return y

def neighbors4(point): 
    "The four neighboring squares."
    x, y = point
    return (          (x, y-1),
            (x-1, y),           (x+1, y), 
                      (x, y+1))

def neighbors8(point): 
    "The eight neighboring squares."
    x, y = point 
    return ((x-1, y-1), (x, y-1), (x+1, y-1),
            (x-1, y),             (x+1, y),
            (x+1, y+1), (x, y+1), (x+1, y+1))

def cityblock_distance(p, q=(0, 0)): 
    "Manhatten distance between two points."
    return abs(X(p) - X(q)) + abs(Y(p) - Y(q))

def distance(p, q=(0, 0)): 
    "Hypotenuse distance between two points."
    return math.hypot(X(p) - X(q), Y(p) - Y(q))

################ Debugging 

def trace1(f):
    "Print a trace of the input and output of a function on one line."
    def traced_f(*args):
        result = f(*args)
        print('{}({}) = {}'.format(f.__name__, ', '.join(map(str, args)), result))
        return result
    return traced_f

def grep(pattern, iterable):
    "Print lines from iterable that match pattern."
    for line in iterable:
        if re.search(pattern, line):
            print(line)

################ A* and Breadth-First Search (we track states, not actions)

def Astar(start, moves_func, h_func, cost_func=lambda s, s2: 1):
    "Find a shortest sequence of states from start to a goal state (a state s with h_func(s) == 0)."
    frontier  = [(h_func(start), start)] # A priority queue, ordered by path length, f = g + h
    previous  = {start: None}  # start state has no previous state; other states will
    path_cost = {start: 0}     # The cost of the best path to a state.
    Path      = lambda s: ([] if (s is None) else Path(previous[s]) + [s])
    while frontier:
        (f, s) = heappop(frontier)
        if h_func(s) == 0:
            return Path(s)
        for s2 in moves_func(s):
            g = path_cost[s] + cost_func(s, s2)
            if s2 not in path_cost or g < path_cost[s2]:
                heappush(frontier, (g + h_func(s2), s2))
                path_cost[s2] = g
                previous[s2] = s

def bfs(start, moves_func, goals):
    "Breadth-first search"
    goal_func = (goals if callable(goals) else lambda s: s in goals)
    return Astar(start, moves_func, lambda s: (0 if goal_func(s) else 1))
def read_input(day):
    target = f'input/day{day}.txt'
    with open(target) as f:
        return f.read().strip()

# Day 1

In [3]:
txt = read_input(1)

In [4]:
txt

'878938232157342756754254716586975125394865297349321236586574662994429894259828536842781199252169182743449435231194436368218599463391544461745472922916562414854275449983442828344463893618282425242643322822916857935242141636187859919626885791572268272442711988367762865741341467274718149255173686839265874184176985561996454253165784192929453678326937728571781212155346592432874244741816166328693958529938367575669663228335566435273484331452883175981955679335327231995452231118936393192583338222595982522833468533262224874637449624644318418748617949417939228988293391941457722641936417456243894182668197174255786445994567477582715692336249243254711653529871336129825735249667425238573952339922948214218872417858525199642194588448543565474847272984232637466664695217176358283788781843171636841215675851778984619377575696447366844854289534215286959727688419731976631323833892247438149829975856161755122857643731945913335556288817112993911694972667656914238999291831997163412548977649491227219477796124134

In [5]:
def solve(in_txt):
    n = len(in_txt)
    sl = [int(in_txt[i:i+1]) for i in range(0, n)]
    el = [(in_txt[i] == in_txt[(i+1) % n]) for i in range(0, n)]
    x = sum(sl[i] * (el[i]==True)  for i in range(0, n))
    
    #print(list(sl[i] * (el[i]==True) for i in range(0, n)))
    return x

In [6]:
solve(txt)

1175

test

In [7]:
solve('1122')

3

In [8]:
solve('1111')

4

In [9]:
solve('1234')

0

In [10]:
solve('91212129')

9

In [11]:
def solve_pt2(in_txt):
    n = len(in_txt)
    sl = [int(in_txt[i:i+1]) for i in range(0, n)]
    #el = [(in_txt[i] == in_txt[(i+1) % n]) for i in range(0, n)]
    el2 = [(in_txt[i] == in_txt[(i+int(n/2)) % n]) for i in range(0, n)]
    #x = sum(sl[i] * (el[i]==True)  for i in range(0, n))
    x = sum(sl[i] * (el2[i]==True)  for i in range(0, n))
    
    return x

In [12]:
solve_pt2('1212')

6

In [13]:
solve_pt2('1221')

0

In [14]:
solve_pt2('123425')

4

In [15]:
solve_pt2('123123')

12

In [16]:
solve_pt2('12131415')

4

In [17]:
solve_pt2(txt)

1166

# Day 2

In [18]:
df = pd.read_csv('input/day2.txt', sep='\t', header=None)

In [19]:
# solve prob 1
x = 0
for i in range(0, len(df)):
    print(i, max(df.iloc[i]) - min(df.iloc[i]))
    x = x + max(df.iloc[i]) - min(df.iloc[i])

x

0 1218
1 6952
2 609
3 458
4 931
5 3896
6 1698
7 2376
8 1096
9 5626
10 393
11 1892
12 5689
13 4095
14 4128
15 830


41887

In [20]:
def evenly_divisible(vec):
    for i in range(0, len(vec)):
        for j in range(0, len(vec)):
            if vec[i] % vec[j] == 0 and vec[i] != vec[j]:
                return max(vec[i], vec[j]) / min(vec[i], vec[j])


In [21]:
evenly_divisible([5,9, 2, 8])

4.0

In [22]:
evenly_divisible([9, 4, 7, 3])

3.0

In [23]:
evenly_divisible([3, 8, 6, 5])

2.0

In [24]:
# solve prob 2
x = 0
for i in range(0, len(df)):
    print(i, evenly_divisible(df.iloc[i]))
    x = x + evenly_divisible(df.iloc[i])

x

0 5.0
1 28.0
2 5.0
3 4.0
4 8.0
5 23.0
6 13.0
7 11.0
8 4.0
9 30.0
10 3.0
11 10.0
12 23.0
13 24.0
14 26.0
15 9.0


226.0

In [32]:
sum(map(evenly_divisible, array(df)))

NameError: name 'array' is not defined

# Day 3

In [44]:
# pt 1
in3 = 368078
def solve3(x):
    i = 1
    while i * i < x:
        i = i + 2

    piv = [i*i - k*(i-1) for k in range(4)]
    for p in piv:
        dist = abs(p - x)
        if dist <= (i-1)//2:
            return(i-1-dist)

In [40]:
solve3(1)

0

In [41]:
solve3(12)

3

In [42]:
solve3(23)

2

In [43]:
solve3(1024)

31

In [45]:
solve3(in3)

371

# Day 4

In [3]:
in4 = read_input(4)

In [36]:
def no_dupes(x):
    y = x.split()
    return len(Set(y)) == len(y) 

In [37]:
sum(no_dupes(x) for x in in4.split("\n"))

466

In [34]:
def no_anagrams(x):
    y = list(map(lambda x: cat(sorted(x)), x.split()))
    return len(Set(y)) == len(y)

In [35]:
sum(no_anagrams(x) for x in in4.split("\n"))

251